<a href="https://colab.research.google.com/github/BooEiEi/Predicting-Technology-Sector-Stock-Prices/blob/main/Hyperparameter_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF4'

Mounted at /content/drive


In [ ]:
import sklearn
#import scikeras
import tensorflow

print("scikit-learn version:", sklearn.__version__)
#print("scikeras version:", scikeras.__version__)
print("tensorflow version:", tensorflow.__version__)


scikit-learn version: 1.6.1
tensorflow version: 2.18.0


In [1]:
!pip uninstall scikit-learn -y

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1


In [2]:
pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 118.9 MB/s eta 0:00:00


In [ ]:
import os
os._exit(00)

In [2]:
!pip install scikeras

In [3]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Dense, Flatten, LSTM, Bidirectional, Input, Dropout, InputLayer, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [4]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [5]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
df_META = df_META.sort_values('date').reset_index(drop=True)

In [6]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
df_AAPL = df_AAPL.sort_values('date').reset_index(drop=True)

In [7]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
df_MSFT = df_MSFT.sort_values('date').reset_index(drop=True)

In [8]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
df_NVDA = df_NVDA.sort_values('date').reset_index(drop=True)

In [9]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])
df_GOOG = df_GOOG.sort_values('date').reset_index(drop=True)

In [10]:
train_size = int(len(df_META) * 0.75)
test_size = len(df_META) - train_size

# META

## Train Test

In [11]:
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [12]:
X_META = train_META[["S&P500","CPI","Interest","US_inflation_rate", "NASDAQ"]]
y_META = train_META["Stockprice_META"]

#Normalized

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

X_META = scaler.fit_transform(X_META)

In [14]:
X_META.shape[0]

564

In [15]:
look_back = 10

X_META1, y_META1 = [], []
for i in range(len(X_META) - look_back):
    X_META1.append(X_META[i:i + look_back])
    y_META1.append(y_META[i + look_back])

X_META1, y_META1 = np.array(X_META1), np.array(y_META1)

In [16]:
y_META1.shape

(554,)

In [17]:
X_META1.shape

(554, 10, 5)

In [18]:
tscv = TimeSeriesSplit(n_splits=5)

#MODEL

In [19]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(LSTM(units=neurons, input_shape=(look_back, X_META1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_META1, y_META1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_META1)
mse = mean_squared_error(y_META1, y_pred)
rmse = root_mean_squared_error(y_META1, y_pred)
mae = mean_absolute_error(y_META1, y_pred)
mape = mean_absolute_percentage_error(y_META1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 256, 'optimizer': 'Adam'}
Test Set Mean Squared Error: 165.7516
Test Set Root Mean Squared Error: 12.8745
Test Set Mean Absolute Error: 9.6628
Test Set Mean Absolute Percentage Error: 0.0406


#APPL

In [20]:
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [21]:
X_AAPL = train_AAPL[["S&P500","CPI", "NASDAQ"]]
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [22]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [23]:
look_back = 10

X_AAPL1, y_AAPL1 = [], []
for i in range(len(X_AAPL) - look_back):
    X_AAPL1.append(X_AAPL[i:i + look_back])
    y_AAPL1.append(y_AAPL[i + look_back])

X_AAPL1, y_AAPL1 = np.array(X_AAPL1), np.array(y_AAPL1)

In [24]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(LSTM(units=neurons, input_shape=(look_back, X_AAPL1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_AAPL1, y_AAPL1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_AAPL1)
mse = mean_squared_error(y_AAPL1, y_pred)
rmse = root_mean_squared_error(y_AAPL1, y_pred)
mae = mean_absolute_error(y_AAPL1, y_pred)
mape = mean_absolute_percentage_error(y_AAPL1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 32, 'epochs': 200, 'neurons': 512, 'optimizer': 'Adam'}
Test Set Mean Squared Error: 67.1403
Test Set Root Mean Squared Error: 8.1939
Test Set Mean Absolute Error: 6.6385
Test Set Mean Absolute Percentage Error: 0.0404


#MSFT

In [25]:
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [26]:
X_MSFT = train_MSFT[["S&P500","CPI","Interest","US_inflation_rate", "NASDAQ"]]
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [27]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [28]:
look_back = 10

X_MSFT1, y_MSFT1 = [], []
for i in range(len(X_MSFT) - look_back):
    X_MSFT1.append(X_MSFT[i:i + look_back])
    y_MSFT1.append(y_MSFT[i + look_back])

X_MSFT1, y_MSFT1 = np.array(X_MSFT1), np.array(y_MSFT1)

In [29]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(LSTM(units=neurons, input_shape=(look_back, X_MSFT1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_MSFT1, y_MSFT1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_MSFT1)
mse = mean_squared_error(y_MSFT1, y_pred)
rmse = root_mean_squared_error(y_MSFT1, y_pred)
mae = mean_absolute_error(y_MSFT1, y_pred)
mape = mean_absolute_percentage_error(y_MSFT1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 512, 'optimizer': 'AdamW'}
Test Set Mean Squared Error: 132.3200
Test Set Root Mean Squared Error: 11.5030
Test Set Mean Absolute Error: 8.8158
Test Set Mean Absolute Percentage Error: 0.0284


#NVDA

In [30]:
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [31]:
X_NVDA = train_NVDA[["S&P500", "BTC", "CPI", "Gold","US_inflation_rate", "NASDAQ"]]
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [32]:
X_NVDA = scaler.fit_transform(X_NVDA)

In [33]:
look_back = 10

X_NVDA1, y_NVDA1 = [], []
for i in range(len(X_NVDA) - look_back):
    X_NVDA1.append(X_NVDA[i:i + look_back])
    y_NVDA1.append(y_NVDA[i + look_back])

X_NVDA1, y_NVDA1 = np.array(X_NVDA1), np.array(y_NVDA1)

In [34]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(LSTM(units=neurons, input_shape=(look_back,X_NVDA1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_NVDA1, y_NVDA1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_NVDA1)
mse = mean_squared_error(y_NVDA1, y_pred)
rmse = root_mean_squared_error(y_NVDA1, y_pred)
mae = mean_absolute_error(y_NVDA1, y_pred)
mape = mean_absolute_percentage_error(y_NVDA1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 512, 'optimizer': 'AdamW'}
Test Set Mean Squared Error: 4.9071
Test Set Root Mean Squared Error: 2.2152
Test Set Mean Absolute Error: 1.4421
Test Set Mean Absolute Percentage Error: 0.0407


#GOOG

In [35]:
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [36]:
X_GOOG = train_GOOG[["S&P500", "Gold", "CPI", "Interest", "UNEMPLOYMENT", "NASDAQ"]]
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [37]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [38]:
look_back = 10

X_GOOG1, y_GOOG1 = [], []
for i in range(len(X_GOOG) - look_back):
    X_GOOG1.append(X_GOOG[i:i + look_back])
    y_GOOG1.append(y_GOOG[i + look_back])

X_GOOG1, y_GOOG1 = np.array(X_GOOG1), np.array(y_GOOG1)

In [39]:
def create_model(neurons=16, optimizer='Adam'):
    model = Sequential()
    model.add(LSTM(units=neurons, input_shape=(look_back,X_GOOG1.shape[2])))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model,
                       neurons=16,
                       optimizer='Adam',
                       verbose=0)


param_grid = {'neurons': [16, 32, 64, 128, 256, 512],
              'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_GOOG1, y_GOOG1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_GOOG1)
mse = mean_squared_error(y_GOOG1, y_pred)
rmse = root_mean_squared_error(y_GOOG1, y_pred)
mae = mean_absolute_error(y_GOOG1, y_pred)
mape = mean_absolute_percentage_error(y_GOOG1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'neurons': 128, 'optimizer': 'Adam'}
Test Set Mean Squared Error: 19.4601
Test Set Root Mean Squared Error: 4.4114
Test Set Mean Absolute Error: 3.5231
Test Set Mean Absolute Percentage Error: 0.0293
